In [3]:
import os
import argparse
import time
from datetime import timedelta
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

from accelerate import Accelerator
from peft import LoraConfig, get_peft_model
from diffusers import StableDiffusionPipeline

In [ ]:
def main():
    # CUDA memory tweaks
    if torch.cuda.is_available():
        torch.cuda.set_per_process_memory_fraction(0.8)
        os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
        torch.cuda.empty_cache()

    args = argparse.Namespace(
        data_dir='data',
        output_dir='./models/lora-checkpoints',
        model_id='runwayml/stable-diffusion-v1-5',
        lora_rank=4,
        num_epochs=10,
        batch_size=1,
        image_size=512,
        max_pairs=60,
        num_workers=2
    )

    print("초기화 시작...")
    accelerator = Accelerator()
    device = accelerator.device
    print(f"사용 장치: {device}")

    # 모델 & 파이프라인 로드 (기본 float32)
    print("모델 로딩 중...")
    pipe = StableDiffusionPipeline.from_pretrained(
        args.model_id,
        use_safetensors=True,
    )

    if torch.cuda.is_available():
        pipe.vae.to(device)
        pipe.text_encoder.to(device)
        pipe.unet.to(device)
        pipe.enable_xformers_memory_efficient_attention()
        print("\n초기 메모리 사용량:")
        print(f"- UNet: {torch.cuda.memory_allocated() / 1024**2:.1f}MB")

    print("모델 로딩 완료")

    tokenizer = pipe.tokenizer
    text_encoder = pipe.text_encoder
    unet = pipe.unet

    # LoRA 설정
    print("LoRA 설정 중...")
    lora_config = LoraConfig(
        r=args.lora_rank,
        lora_alpha=4,
        target_modules=["to_k", "to_q", "to_v", "to_out.0"],
        bias="none"
    )
    unet = get_peft_model(unet, lora_config)
    print("LoRA 설정 완료")

    # 데이터셋 & 데이터로더
    print("데이터셋 생성 중...")
    dataset = PosterPairDataset(args.data_dir, tokenizer, args.image_size, args.max_pairs)
    train_dataloader = DataLoader(
        dataset,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=args.num_workers,
        pin_memory=True,
        persistent_workers=True
    )
    print(f"데이터셋 크기: {len(dataset)}")
    print(f"배치 크기: {args.batch_size}")

    # 옵티마이저
    optimizer = torch.optim.AdamW(unet.parameters(), lr=1e-5)

    # 학습 루프
    print("학습 시작...")
    unet.train()
    start_time = time.time()
    print_interval = 100

    if torch.cuda.is_available():
        print("\n학습 시작 전 메모리 사용량:")
        print(f"- 할당된 메모리: {torch.cuda.memory_allocated() / 1024**2:.1f}MB")
        print(f"- 캐시된 메모리: {torch.cuda.memory_reserved() / 1024**2:.1f}MB")
        print(f"- 배치 크기 {args.batch_size}로 학습을 시작합니다.")

    total_batches = len(train_dataloader) * args.num_epochs
    print(f"\n총 배치 수: {total_batches}, 총 에폭: {args.num_epochs}")

    for epoch in range(args.num_epochs):
        print(f"Epoch {epoch+1}/{args.num_epochs}")
        for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            pixel_values = batch["pixel_values"].to(device)
            target_pixel_values = batch["target_pixel_values"].to(device)
            input_ids = batch["prompt_ids"].to(device)

            # VAE 인코딩 (horizontal 이미지)
            with torch.no_grad():
                latents = pipe.vae.encode(target_pixel_values).latent_dist.sample()
                latents = latents * pipe.vae.config.scaling_factor
                latents = latents.to(device)

            timesteps = torch.randint(
                0, pipe.scheduler.config.num_train_timesteps,
                (latents.shape[0],), device=device
            )
            noise = torch.randn_like(latents)
            noisy_latents = pipe.scheduler.add_noise(latents, noise, timesteps)

            text_embeddings = text_encoder(input_ids)[0]

            optimizer.zero_grad()
            model_pred = unet(noisy_latents, timesteps, text_embeddings).sample
            loss = F.mse_loss(model_pred, noise)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(unet.parameters(), 1.0)
            optimizer.step()

            # NaN 체크
            if torch.isnan(loss):
                print(f"NaN loss at step {step}, breaking.")
                return

            # 로그 출력
            if step % print_interval == 0:
                elapsed = time.time() - start_time
                ips = (step + 1) / elapsed
                remaining = (total_batches - (step + 1)) / ips
                print(f"\nStep {step}/{total_batches} - Loss: {loss.item():.4f}")
                print(f"  경과: {timedelta(seconds=int(elapsed))}, 남은: {timedelta(seconds=int(remaining))}")
                if torch.cuda.is_available():
                    print(f"  GPU 메모리: {torch.cuda.memory_allocated() / 1024**2:.1f}MB")

    # LoRA 가중치 저장
    os.makedirs(args.output_dir, exist_ok=True)
    unet.save_pretrained(args.output_dir)
    print(f"LoRA weights saved to {args.output_dir}")


In [7]:
main()

초기화 시작...
사용 장치: cuda
모델 로딩 중...


Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  5.31it/s]



초기 메모리 사용량:
- UNet: 4133.7MB
모델 로딩 완료
LoRA 설정 중...
LoRA 설정 완료
데이터셋 생성 중...
데이터셋 로딩 시작: ../../data/pairs


영화 폴더 로딩:  87%|████████▋ | 39/45 [00:00<00:00, 5239.52it/s]


데이터셋 로딩 완료: 60개 페어
데이터셋 크기: 60
배치 크기: 4
학습 시작...

학습 시작 전 메모리 사용량:
- 할당된 메모리: 4135.2MB
- 캐시된 메모리: 4184.0MB
- 배치 크기 4로 학습을 시작합니다.

학습 정보:
- 전체 데이터 수: 60
- 배치 크기: 4
- 총 배치 수: 150
- 총 에폭 수: 10
Epoch 1/10


  0%|          | 0/15 [00:05<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. 4.80 GiB allowed; Of the allocated memory 4.67 GiB is allocated by PyTorch, and 45.06 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [10]:
!ls

colab_train_lora.ipynb
